# Recognition of abnormality from Breast Thermograms Using ConvXGB

# Pre - trained ResNET - 50

## Table of Contents
- [1 - Packages](#1)
- [2 - Test and Train Data](#2)
- [3 - ResNET Model](#3)
- [4 - Adam Optimizer](#4)
- [5 - Saving the Best Model](#7)

<a name='1'></a>
# 1 - Packages

In [19]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline
from sklearn.utils import shuffle
import cv2

from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint

<a name='2'></a>
# 2 - Test and Train Data

In [26]:
path_test = "C:/Users/rsvmu/Downloads/Data_2"
CATEGORIES = ["Withcancer","Withoutcancer"]
IMG_SIZE_X = 480
IMG_SIZE_Y = 640

In [27]:
training = []
def createTrainingData():
    for category in CATEGORIES:
        path = os.path.join(path_test, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img)) #Changing RGB to Gray_Scale 
#             aniso = anisodiff(img_array,) # Applying anistropic Diffusion Filter to Gray_scale Images
            new_array = cv2.resize(img_array, (IMG_SIZE_X, IMG_SIZE_Y))
            training.append([new_array, class_num])
createTrainingData()

In [34]:
X =[]
y =[]
for features, label in training:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1,IMG_SIZE_X, IMG_SIZE_Y,3)

In [35]:
X = X.astype('float32')
X /= 255
from keras.utils import np_utils
Y = np_utils.to_categorical(y, 2)
print(Y[5])
print(Y.shape)

[1. 0.]
(215, 2)


In [36]:
X.shape

(215, 480, 640, 3)

In [37]:
from sklearn.model_selection import train_test_split

In [38]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 4)

# # Normalize image vectors
# X_train = X_train_orig/255.
# X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
# Y_train = convert_to_one_hot(Y_train_orig, 6).T
# Y_test = convert_to_one_hot(Y_test_orig, 6).T

# print ("number of training examples = " + str(X_train.shape[0]))
# print ("number of test examples = " + str(X_test.shape[0]))
# print ("X_train shape: " + str(X_train.shape))
# print ("Y_train shape: " + str(Y_train.shape))
# print ("X_test shape: " + str(X_test.shape))
# print ("Y_test shape: " + str(Y_test.shape))

In [39]:
Y_train.shape

(161, 2)

<a name='3'></a>
# 3 - ResNET Model Model

In [40]:
import tensorflow

In [41]:
img_height,img_width = 480,640 
num_classes = 2
#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = tensorflow.keras.applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,3))

In [42]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

<a name='4'></a>
# 4 - Adam Optimizer

In [43]:
from tensorflow.keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='binary_crossentropy', metrics=['accuracy'])

<a name='5'></a>
# 5 -Saving the Best Model

In [44]:
X_train = tensorflow.stack(X_train)
Y_train = tensorflow.stack(Y_train)

In [ ]:
model.fit(X_train, Y_train, verbose=0, epochs=2)

In [ ]:
X_train.shape

In [ ]:
Y_train.shape

In [ ]:
Y.shape